<a href="https://colab.research.google.com/github/CMallart/ateliers-NN/blob/main/TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
from sklearn.datasets import load_wine
from sklearn.model_selection import KFold

# usefull functions

In [ ]:
def one_hot_encode_database(database, list_id):
  """One hot encode the columns of the database specified in list_id

  return a copy of the database, on hot encoded
  """
  encoded_database = np.empty(shape=(database.shape[0], 0), dtype=float)
  for id in range(database.shape[1]):
      if id in list_id:
          original_column = database[:, id]
          encoded_column = to_categorical(original_column.astype(int), num_classes=int(np.max(original_column)+1))
          encoded_database = np.column_stack((encoded_database, encoded_column))
      else:
          original_column = database[:, id]
          encoded_database = np.column_stack((encoded_database, original_column))
  return encoded_database

In [ ]:
def normalize_database(database, list_id):
  """Normalize between 0 and 1 each column of the database specified in list_id.

  return a copy of the database, normalized. 
  """
  encoded_database = database.copy()
  for id in list_id:
      encoded_database[:, id] = (encoded_database[:, id] - np.amin(encoded_database[:, id])) / (np.amax(encoded_database[:, id]) - np.amin(encoded_database[:, id]))
  return encoded_database

In [ ]:
def build_NN(layer_list, input_dim, output_dim, lr=0.001):
  """Construct a layer composed of dense layers, which dimensions are definded in the layer_list argument.

  return the constructed and compiled model.
  """
  # Q7: add layers and "compile" the model
  model = Sequential()
  #Add layers
  model.add()
    
  #Compile the network
  model.compile(loss=None, optimizer=None, metrics=None)
  return model

# Main

## Hyper-parameters

In [ ]:
nb_fold = 5
layer_list = [50, 50, 20]

## Initialization

In [ ]:
rng = np.random.default_rng()

# Hyper-parameters

In [ ]:

database = load_wine()
print('--- original database ---\n')
print(f'{database.data.shape[0]} examples, {database.data.shape[1]} features')
for i in [10, 80, 140]:
  print(f'label of example {i:3d}: {database.target[i]}')

# Q7: one hot encode the class and normalize the features of the database here

random_indices = np.arange(data.shape[0])
rng.shuffle(random_indices)
data = data[random_indices,:]
target = target[random_indices,:]
print('\n--- after randomization ---\n')
print(f'{data.shape[0]} examples, {data.shape[1]} features')
for i in [10, 80, 140]:
  print(f'label of example {i:3d}: {target[i]}')

--- original database ---

178 examples, 13 features
label of example  10: 0
label of example  80: 1
label of example 140: 2


NameError: ignored

## Build / Fit / avaluate (quick check) 

In [ ]:
# Q10: quick check
model = build_NN(layer_list, data.shape[1], target.shape[1], lr=0.1)
model.summary()
model.fit(data, target, batch_size=16, epochs=100)
model.evaluate(data, target)

## Build / Fit / avaluate (cross-validated) 

In [ ]:
gt_and_pred = np.empty(shape=(2, 0))

for train_index, test_index in KFold(n_splits=nb_fold).split(data):
  training_data, validation_data = data[train_index], data[test_index]
  training_target, validation_target = target[train_index], target[test_index]

  # Q11: test several learning rates
  model = build_NN(layer_list, data.shape[1], target.shape[1], lr=0.1)

  callback = EarlyStopping(patience=5)
  # Q13: add EarlyStopping() callback
  model.fit(training_data, training_target, batch_size=16, epochs=200, validation_data=(validation_data, validation_target), shuffle=True)

  fold_prediction = model.predict(validation_data)
  gt_and_pred = np.column_stack((gt_and_pred, np.array([np.argmax(validation_target, axis=1), np.argmax(fold_prediction, axis=1)])))

In [ ]:
# Q12: display confusion matrix and balanced accuracy here